# Electronic Structure Calculations Using Python

## Library Installations

In [1]:
!pip install pyscf
!pip install matplotlib==3.1.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 25.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 44.6 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2


## Primer on Electronic Structure

Electronic structure strives to obtain molecular properties based on a quantum mechanical description of chemical systems. Many methods exist with varying degree of accuracy and computational demands. This tutorial will describe the Hartree-Fock framework as the method to obtain an initial state and later the coupled cluster formalism to refine this initial state and obtain a more accurate electronic energy.

The electronic Hamiltonian is comprised of 1-electron kinetic energy, 2-electron repulsion, the nuclear-electronic potential term and the nuclear-nuclear potential, which is typically a constant under the Born-Oppenheimer approximation.

\begin{align}
	\hat{H} = h_{1e} + h_{2e} + h_{nuc}
\end{align}

Where, $h_{1e}$ contains the 1-electron terms, $h_{2e}$ contains the 2-electron terms and the $h_{nuc}$ contains the electron-proton attraction. Each of these terms require specification of a molecular structure and a basis set. In the next section, the procedure is shown to perform a calculation on a classical computer.


### Electronic Structure in a Classical Computer

This section exposes the methods containing the integrals within the PySCF electronic structure code. These are calculated for a given choice of molecular geometry, basis, charge and spin state. The following contains the same example of $H_2$ with a STO3-G basis, charge = 0 with both electrons paired. An Hartree-Fock electronic structure calculation example is also shown.

#### PySCF specific imports

These modules are required to generate a molecule object, `gto`, perform a self-consistent field calculation, `scf`, and convert integral matrices from an atomic orbital description to a molecular orbital description, `ao2mo`.

In [2]:
from pyscf import gto, scf
# from pyscf.scf import _vhf
from pyscf import ao2mo

In the application to the $H_2$ case, the following two lines use the method `M` of the class [`gto`](https://pyscf.org/user/gto.html) from PySCF, documented in ([gto](https://pyscf.org/pyscf_api_docs/pyscf.gto.html?highlight=pyscf+gto#module-pyscf.gto)) to define the molecule object. The molecular geometry is defined via `atom`, and the basis set with the `basis` keyword. By default, `charge=0` and `spin=0` implying a neutral species in the singlet state.

In [3]:
molecule = gto.M(atom='H 0 0 0; H 0 0 0.7414', basis='sto3g')

#### Integrals - AO basis

This section focuses on obtaining the integrals in an atomic orbital basis. The object instantiated from the class `gto` supports the method `intor` which allows access to the integrals is documented (https://pyscf.org/pyscf_api_docs/pyscf.gto.html?highlight=intor#pyscf.gto.mole.Mole.intor ) and allows specification of different types of integrals. In the following example we calculate each of these integrals, required by the Hartree-Hock method for the $H_2$ molecule as defined in the previous cell. Each integral equation is shown alongside the code element that calculates it.

The nuclear-electronic coulombic attraction integral for the electron is given by the expectation value of the operator:

\begin{equation}
	\hat{h}_{eN} = - \sum_{a}^{N_{nuclei}} \frac{Z_a}{|\vec{R}_a-\vec{r}|}
\end{equation}

And the expectation value itself:

\begin{equation}
	\langle \phi_i |\hat{h}_{eN} | \phi_j \rangle = - \langle \phi_i | \sum_{a}^{N_{nuclei}} \frac{Z_a}{|\vec{R}_a-\vec{r}|} | \phi_j \rangle = \int d\vec{x} \phi_i^*(\vec{x}) \left(\sum_{a}^{N_{nuclei}} \frac{Z_a}{|\vec{R}_a-\vec{r}|}\right)\phi_j(\vec{x})
\end{equation}

Where $Z_a$ is the nuclear charge, $R_a$ is the nuclear position and $r$ is the electron position

In [4]:
nuclear_1e = molecule.intor('int1e_nuc')
print("\n Nuclear-electronic coulombic attraction: \n", nuclear_1e)


 Nuclear-electronic coulombic attraction: 
 [[-1.88008303 -1.19385819]
 [-1.19385819 -1.88008303]]


The electronic kinetic energy integral for the electron is given by the expectation value of the operator:

\begin{equation}
	\hat{h}_{e,kin} = - \frac{1}{2} \nabla^2 
\end{equation}

And the expectation value itself:

\begin{equation}
	\langle \phi_i |\hat{h}_{e,kin} | \phi_j \rangle = - \langle \phi_i | \frac{1}{2} \nabla^2 | \phi_j \rangle = \int d\vec{x} \phi_i^*(\vec{x}) \left(-\frac{1}{2} \nabla^2 \right)\phi_j(\vec{x})
\end{equation}

In [5]:
kinetic_1e = molecule.intor('int1e_kin')
print("\n Electronic kinetic energy: \n", kinetic_1e)


 Electronic kinetic energy: 
 [[0.76003188 0.23612597]
 [0.23612597 0.76003188]]


The electronic kinetic energy integral for electron $i$ is given by the expectation value of the operator:

\begin{equation}
	\hat{h}_{ee} = \frac{1}{|{r}_1-{r}_2|}
\end{equation}

And the expectation value itself:

\begin{equation}
	\langle \phi_i \phi_j|\hat{h}_{ee} | \phi_k \phi_l \rangle = \langle \phi_i\phi_j | \frac{1}{|{r}_1-{r}_2|} | \phi_k \phi_l\rangle = \int d\vec{x}_1 \int d\vec{x}_2 \phi_i^*(\vec{x})\phi_j^*(\vec{x}) \left(\frac{1}{|{r}_1-{r}_2|} \right)\phi_k(\vec{x})\phi_l(\vec{x})
\end{equation}

This would yield a tensor containing all possible values for the indices $i$ and $j$, as indicated on the cell below.

In [6]:
coul_2e_full_ao = molecule.intor('int2e', aosym='s1')
import numpy as np
print("\n Electronic-electronic coulombic repulsion is a tensor of dimensions n^4 \n", np.shape(np.round(coul_2e_full_ao, decimals=6)))
print("\n Electronic-electronic coulombic repulsion, no-symmetry: \n", 
        coul_2e_full_ao)


 Electronic-electronic coulombic repulsion is a tensor of dimensions n^4 
 (2, 2, 2, 2)

 Electronic-electronic coulombic repulsion, no-symmetry: 
 [[[[0.77460594 0.44379315]
   [0.44379315 0.56946841]]

  [[0.44379315 0.29666317]
   [0.29666317 0.44379315]]]


 [[[0.44379315 0.29666317]
   [0.29666317 0.44379315]]

  [[0.56946841 0.44379315]
   [0.44379315 0.77460594]]]]


In practice, symmetry can be exploited such that few integrals are calculated, allowing for time/memory savings.

In [7]:
coul_2e_sym_ao = molecule.intor('int2e', aosym='s8')
print("\n Electronic-electronic coulombic repulsion: \n", coul_2e_sym_ao)


 Electronic-electronic coulombic repulsion: 
 [0.77460594 0.44379315 0.29666317 0.56946841 0.44379315 0.77460594]


The nuclear-nuclear integral is given by the expression.

\begin{equation}
	\hat{h}_{eN} = \sum_{a}^{N_{nuclei}}\sum_{b > a}^{N_{nuclei}} \frac{Z_a Z_b}{|{R}_a-{R}_b|}
\end{equation}

In the Born-Oppenheimer approximation, the nuclei are static with respect to the fast-moving electrons and thus this term is typically a constant added to the electronic energy.

In [8]:
nuclear_rep = molecule.get_enuc()
print("\n Nuclear-nuclear coulombic repulsion", nuclear_rep)


 Nuclear-nuclear coulombic repulsion 0.7137539936876182


The so-called core hamiltonian is comprised of one-electron terms, that is:

\begin{equation}
	\hat{h}_{core} = \hat{h}_{e,kin} + \hat{h}_{eN}
\end{equation}

In [9]:
hcore_ao = molecule.get_hcore()
print("\n Core hamiltonian, sum of 1 electron terms \n", hcore_ao)

converged SCF energy = -1.11668438708534

 Core hamiltonian, sum of 1 electron terms 
 [[-1.12005114 -0.95773222]
 [-0.95773222 -1.12005114]]


### The Electronic Structure Problem
Solving the Hartree-Fock electronic structure problem using the PySCF package and collecting electronic energy, molecular orbital coefficients and molecular orbital energies. The RHF method is being used from the self-consistent field module, using the previously defined molecule object as argument.

In [10]:
# Electronic structure calculation
RHF_calc = scf.RHF(molecule)

This cell runs the calculation and **updates** the atributes of the `RHF_calc` object. All the electronic structure observables supported by PySCF populate the associated attributes.

In [11]:
scf_result = RHF_calc.kernel()

converged SCF energy = -1.11668438708534


Calling the `scf_result` after the calculation execution yields only the electronic energy, given by the following expression:

\begin{equation}
    E = \sum_{i}^{N_elec} \langle \phi_i|\hat{h}_i| \phi_i\rangle + \frac{1}{2} \sum_{ij}^{N_{elec}} (\langle \phi_i \phi_j|\hat{h}_{ee}| \phi_i \phi_j \rangle - \langle \phi_i \phi_j|\hat{h}_{ee}| \phi_j \phi_i \rangle) + V_{NN}
\end{equation}

In [12]:
print("\n Reference Electronic Energy: \n", scf_result)


 Reference Electronic Energy: 
 -1.1166843870853405


However, other attributes can be accessed, including the molecular orbital expansion in the atomic orbital basis, via `mo_coeff`, as well as the molecular orbital energies, via `mo_en`. Do note that `mo_coeff` is a matrix of size $n$ by $n$, where $n$ is the number of basis functions, with each column being a different molecular orbital and each row an atomic orbital.

In [13]:
# MO coefficients and energies
mo_coeff = RHF_calc.mo_coeff
mo_en = RHF_calc.mo_energy
print("\n Molecular Orbital coefficients in AO basis: \n", mo_coeff)
print("\n Molecular Orbital Energies: \n", mo_en)


 Molecular Orbital coefficients in AO basis: 
 [[ 0.54899378  1.21082257]
 [ 0.54899378 -1.21082257]]

 Molecular Orbital Energies: 
 [-0.57797481  0.66969867]


Solving the Hartree-Fock generalized eigenvalue problem manually, from the Fock matrix and the overlap matrix, requires solving the generalized eigenvalue problem:

\begin{equation}
    FC = SCE
\end{equation}

Where $F$ is the Fock matrix, $S$ is the overlap matrix, $C$ is the molecular orbital coefficient matrix and $E$ is the energy vector. The Fock matrix is defined:

\begin{equation}
    F = h + G\cdot D 
\end{equation}

Where $h$ is the one-electron matrix, $D$ is the density matrix and $G$ is the 4 dimensional tensor containing the two electron integrals The calculation proceeds iteratively until finding the optimal density that minimizes the Hartree-Fock energy.

The Fock and overlap matrices can be obtained by accessing the the `get_fock` and `get_ovlp` methods from the `RHF_calc` object.

In [14]:
# Relevant matrices for generalized eigenvalue equation FC = SCE
fock = RHF_calc.get_fock()
ovlp = RHF_calc.get_ovlp()
print("\n Fock matrix: \n", fock)
print("\n Overlap matrix: \n", ovlp)


 Fock matrix: 
 [[-0.36521973 -0.59361569]
 [-0.59361569 -0.36521973]]

 Overlap matrix: 
 [[1.         0.65895712]
 [0.65895712 1.        ]]


Solving the generalized eigenvalue problem requires recasting the above equation:

\begin{align}
    FC = SCE \equiv S^{-1}FC = CE
\end{align}

The following function harnesses the `numpy` library to perform matrix algebra and order the eigenvalues and corresponding eigenvectors for a given matrix, `A`.

In [15]:
import numpy as np

def eigen(A):
    '''
        Given a matrix A, returns ordered eigenvalues and corresponding eigenvectors
        Eigenvectors are in column format
    '''
    eigenValues, eigenVectors = np.linalg.eig(A)
    idx = np.argsort(eigenValues)[::]
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]
    return (eigenValues, eigenVectors)

By calling the `eigen` function on the product $S^{-1}F$, the molecular orbital energies `E` and molecular orbital coefficient expansion `C`.

In [16]:
geneigen_mo_en, geneigen_mo_coeff = eigen(np.linalg.inv(ovlp) @ fock)
print("Based on the generalized eigenvalue problem approach, ")
print("\n Molecular Orbital coefficients in AO basis: \n", geneigen_mo_coeff)
print("\n Molecular Orbital Energies: \n", geneigen_mo_en)

Based on the generalized eigenvalue problem approach, 

 Molecular Orbital coefficients in AO basis: 
 [[ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]]

 Molecular Orbital Energies: 
 [-0.57797481  0.66969867]
